<a href="https://colab.research.google.com/github/javacatknight/beginnerLeetcodeSolutions/blob/main/CSC373_EC_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Colab

Colab is an online interface where you can write and run code similar to a Jupyter Notebook.

To run a cell containing code - press the play button that appears on the left hand side of the cell when you mouse hovers over the cell.

**If you want to save your progress**, you should first make a copy of this notebook, and work on the copy.

First things first, run the next cell (which has code hidden) to set up some helper functions!

In [ ]:
# @title Run this cell to set up some helper functions
import graphviz
import plotly.express as px

import random
import itertools
import timeit
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
from dataclasses import dataclass

pio.templates.default = "plotly_white"

def plot_tree(tree):
    plot = graphviz.Digraph()
    def traverse(t, id):
        if t.left is not None:
            left_id = 2*id
            plot.node(str(left_id), str(t.left.value))
            plot.edge(str(id), str(left_id) , "0")
            traverse(t.left, left_id)
        if t.right is not None:
            right_id = 2*id + 1
            plot.node(str(right_id), str(t.right.value))
            plot.edge(str(id), str(right_id), "1")
            traverse(t.right, right_id)
    plot.node("1", str(tree.value))
    traverse(tree, 1)
    return plot

def get_random_points(n):
    return [
        Point(
            random.uniform(0, 1),
            random.uniform(0, 1),
        )
        for _ in range(n)
    ]

def plot_parts(parts, colors):
    fig = go.Figure()
    for points, c in zip(parts, colors):
        xs = [p.x for p in points]
        ys = [p.y for p in points]
        fig.add_trace(go.Scatter(x=xs, y=ys, mode="markers", marker=dict(color=c)))
    fig.update_layout(
        width=800, height=800, showlegend=False
    )
    fig.update_xaxes(range=[-0.1,1.1])
    fig.update_yaxes(range=[-0.1,1.1])
    return fig

def plot_points(points):
    return plot_parts([points], ['blue'])

def plot_points_and_closest(points, closest):
    return plot_parts([points, closest], ['blue', 'red'])

def time(function, input, number):
    return timeit.timeit(lambda: function(input), number=number)

def compare_function_plot(functions, input_generator, sizes, number=5):
    fig = go.Figure()
    inputs = [input_generator(s) for s in sizes]
    for f in functions:
        name = f.__name__
        vals = [time(f, i, number) for i in inputs]
        fig.add_trace(go.Scatter(x=sizes, y=vals, name=name))
    fig.update_xaxes(title_text="Input Size")
    fig.update_yaxes(title_text="Time")
    return fig

# Huffman Coding

In the first part, we will implement Huffman Coding.
1. The cell below contains some text that we will use to learn the encoding - the content of the text is **not** important.
2. `preprocess(text)` converts `text` into lowercase, and keeps only characters from the alphabet, space, comma, period, and dash. We are doing this to make the tree less cluttered when we visualize it later. You can return to this cell later and remove the preprocessing step to get an encoding that covers more punctuation, numbers and uppercase letters if you want.


In [ ]:
# text comes from https://en.wikipedia.org/wiki/Huffman_coding

text = """In 1951, David A. Huffman and his MIT information theory classmates were given the choice of a term paper or a final exam. The professor, Robert M. Fano, assigned a term paper on the problem of finding the most efficient binary code. Huffman, unable to prove any codes were the most efficient, was about to give up and start studying for the final when he hit upon the idea of using a frequency-sorted binary tree and quickly proved this method the most efficient.[jz]

In doing so, Huffman outdid Fano, who had worked with Claude Shannon to develop a similar code. Building the tree from the bottom up guaranteed optimality, unlike the top-down approach of Shannon–Fano coding.

Terminology
Huffman coding uses a specific method for choosing the representation for each symbol, resulting in a prefix code (sometimes called "prefix-free codes", that is, the bit string representing some particular symbol is never a prefix of the bit string representing any other symbol). Huffman coding is such a widespread method for creating prefix codes that the term "Huffman code" is widely used as a synonym for "prefix code" even when such a code is not produced by Huffman's algorithm."""

def preprocess(text):
    return "".join(x for x in text.lower() if x.isalpha() or x in " .,-")

text = preprocess(text)
print(text)

The Counter object provided by Python conveniently counts the frequencies of each letter in `text`

In [ ]:
from collections import Counter
frequencies = Counter(text)
frequencies

In [ ]:
len(frequencies)

The number of distinct characters to encode is 30 so a naive encoding can encode each letter as a sequence of 5 bits.

In [ ]:
# @title Run to visualize `frequencies`
chars, freqs = zip(*reversed(sorted(frequencies.items(), key=lambda x:x[1])))
fig = px.histogram(x=chars, y=freqs)
fig.update_layout(
    xaxis_title="Character",
    yaxis_title="Frequency",
)

As one might expect, the distribution of the letters is not very uniform, with some letters like "i" and "e" occurring often and other letters like "x" and "z" occurring only a few times. Huffman coding exploits this variation to find a more efficient encoding.

**Task.** Implement the Huffman Coding algorithm described in lecture and apply it to the given text. To help you get started, here are a few definitions.

**Binary Tree**
* `BinaryTree(value, left, right)` defines a Binary Tree with label `value`, left child `left`, and right child `right`.
* `left` (resp. `right`) should either be a `BinaryTree` object or `None`. `None` indicates that there is no left (resp. right) child.
* If `tree` is a BinaryTree, then you can visualize it using `plot_tree(tree)`.

**Priority Queues**
* To add something to the priority queue do `q.put(PriortizedItem(priority, item))` where `priority` is the priority and `item` is the item you want to enqueue.
* `q.get()` returns the PriortizedItem with the **smallest** priority
* If `i` is a PrioritizedItem, you can extract the priority and the item itself with `i.priority`, and `i.item`


In [ ]:
class BinaryTree:
    def __init__(self, value, left, right):
        self.value = value
        self.left = left
        self.right = right

from dataclasses import dataclass, field
from typing import Any

# Reference: https://docs.python.org/3/library/queue.html#queue.PriorityQueue
@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)


Example usage of `BinaryTree`

In [ ]:
l = BinaryTree(123, BinaryTree('leaf', None, None), None)
r = BinaryTree(321, None, None) # leaf
p = BinaryTree("example", l, r)
plot_tree(p)

In [ ]:
plot_tree(p.left)

Example usage of PriorityQueue

In [ ]:
import queue
q = queue.PriorityQueue()
q.put(PrioritizedItem(2, "bigger"))
q.put(PrioritizedItem(1, "smaller"))
smaller = q.get()
bigger = q.get()
print(smaller.item)
print(bigger.item)

## Task 1
Find the encoding tree for `text` using the Huffman Coding Algorithm. Once you found it, plot the tree!

In [ ]:
### YOUR CODE HERE

###

Call `extract_encoding` on your encoding tree to get a dictionary mapping characters to their encoding. Store the resulting dictionary in a variable called `encoding`

In [ ]:
def extract_encoding(tree):
    encoding = dict()
    def _extract(t, s):
        if t.left is None and t.right is None:
            encoding[t.value] = s
        if t.left is not None:
            _extract(t.left, s + "0")
        if t.right is not None:
            _extract(t.right, s + "1")
    _extract(tree, "")
    return encoding

encoding = extract_encoding(tree) # replace tree with variable storing your encoding tree
print(encoding)

Let's see how the this encoding compares with the naive encoding. First, let's compare it on the text we learned the encodings from in the first place.

In [ ]:
def encode(encoding, s):
    return "".join([encoding[c] for c in s])

def huffman_encoding_length(encoding, text):
    return len(encode(encoding, text))

def naive_encoding_length(text):
    return len(text) * 5 # since there are 30 < 2^5 characters, the naive encoding uses 5 bits per character

def compare_encoding_length(encoding, text):
    print("Naive Encoding Length: ", naive_encoding_length(text))
    print("Huffman Encoding Length: ", huffman_encoding_length(encoding, text))
    print()

compare_encoding_length(encoding, text)

As you'd expect, Huffman is more efficient! Now, replace the following string with some other random text you can think of to compare the performance on new text!

In [ ]:
compare_encoding_length(encoding, preprocess("Huffman Coding is awesome"))

## How to submit
Change `text` to some other text of your choosing. Run the Huffman algorithm again on that text.

Submit a picture/screenshot of the encoding tree that you get and report the naive encoding length and Huffman encoding length.

## Question
Is the Huffman encoding always shorter than the naive encoding?

# Closest Pair

In this part - we'll implement the closest pair of points in $\mathbb{R}^2$ algorithm from week 1.

Below we provide a `Point` object, as well as a function to calculate the distance between two points.

In [ ]:
@dataclass
class Point:
    x: float
    y: float

def distance(p1, p2):
    x1, y1 = p1.x, p1.y
    x2, y2 = p2.x, p2.y
    return ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5

Here is some example usage, including how to
* Declare points
* Access coordinates
* Compute the distance

In [ ]:
p1 = Point(2, 3)
p2 = Point(3, 9)
print(p1)
print(p1.x)
print(p2.y)
print(distance(p1, p2))

Let's work with a random collection of points

In [ ]:
points = get_random_points(50)
plot_points(points)

Here is a brute force algorithm - it enumerates all pairs of points. You can use this to check your solution for correctness. It will also be fun later to compare the runtime of the divide and conquer algorithm with this one.

In [ ]:
def closest_pair_brute_force(points):
    """Finds the closest pair

    Args:
        points (list): a list of points in R^2

    Returns: (pair, d)
        pair (list): is a list of two points with the smallest distance
        d (float): the smallest distance
    """
    distances = [distance(*pair) for pair in itertools.combinations(points, 2)]
    return min(zip(itertools.combinations(points, 2), distances), key=lambda t: t[1])

closest, dist = closest_pair_brute_force(points)
plot_points_and_closest(points, closest)

## Task 2
Implement the closest pair divide and conquer algorithm below. Feel free to refer back to the slides! Your function should have the inputs and outputs as specifed in the docstring - i.e. it should have the same inputs and outputs as the brute force algorithm.

In [ ]:
def closest_pair(points):
    """Finds the closest pair

    Args:
        points (list): a list of points in R^2

    Returns: (pair, d)
        pair (list): is a list of two points with the smallest distance
        d (float): the smallest distance
    """
    ### YOUR CODE HERE
    pass
    ###

closest_pair(points)

In [ ]:
plot_points_and_closest(points, closest_pair(points)[0])

Optional: Run the next cell to compare the runtime of the divide and conquer algorithm and the brute force algorithm (it might take some time to run)

In [ ]:
compare_function_plot(
    [closest_pair, closest_pair_brute_force],
    get_random_points,
    list(range(2, 1000, 50)),
)

Let's try it with more points! Note that in the plots below you can zoom into a box by clicking and dragging with your mouse.

In [ ]:
random.seed(a="replace with your groups UTorIDs", version=2)
many_points = get_random_points(50000)
plot_points(many_points)

In [ ]:
closest, dist = closest_pair(many_points)
plot_points_and_closest(many_points, closest)

In [ ]:
print(closest, dist)

## How to submit
Change the seed to your groups UTorIDs and submit screenshots of the previous three cells with their outputs